<a href="https://colab.research.google.com/github/DeviPrasad777/CSA0383-DS/blob/main/Cryptocurrency_price_prediction_using_ML_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('crypto_data.csv')
prices = data['Close'].values.reshape(-1, 1)  # Assuming 'Close' is the column with the price data

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 60  # Sequence length for CNN
X, y = create_sequences(scaled_prices, seq_length)

# Reshape X for Conv1D
X = X.reshape((X.shape[0], X.shape[1], 1))

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_length, 1)),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, batch_size=32)

# Revert to original price data for ARIMA
arima_prices = prices.flatten()

# Fit ARIMA model (You may need to choose appropriate p, d, q values)
model_arima = ARIMA(arima_prices, order=(5, 1, 0))
model_arima_fit = model_arima.fit()

# Forecast future prices
arima_forecast = model_arima_fit.forecast(steps=10)

# CNN prediction for the last sequence in the data
last_sequence = scaled_prices[-seq_length:]
cnn_prediction = model.predict(last_sequence.reshape(1, seq_length, 1))
cnn_prediction = scaler.inverse_transform(cnn_prediction)

# CNN prediction for the last sequence in the data
last_sequence = scaled_prices[-seq_length:]
cnn_prediction = model.predict(last_sequence.reshape(1, seq_length, 1))
cnn_prediction = scaler.inverse_transform(cnn_prediction)

# ARIMA and CNN predictions
plt.figure(figsize=(10, 6))
plt.plot(arima_forecast, color='blue', label='ARIMA Prediction')
plt.plot(range(len(arima_prices), len(arima_prices) + 1), cnn_prediction.flatten(), color='red', label='CNN Prediction')
plt.legend()
plt.show()



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten

# Load your dataset
# Assuming the data is in a CSV file and has a 'Close' column for closing prices
data = pd.read_csv('cryptocurrency_data.csv')
data = data['Close'].values
data = data.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Prepare the dataset
lookback = 60  # The number of previous time steps to consider for predicting the next time step
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build the model
model = Sequential()

# CNN Layer
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

# LSTM Layer
model.add(Reshape((29, 64)))  # Reshaping before feeding into LSTM
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=20, batch_size=32)

# Prediction
predicted_prices = model.predict(X)
predicted_prices = scaler.inverse_transform(predicted_prices)

# Plot the results
plt.plot(data, color='blue', label='Actual Cryptocurrency Price')
plt.plot(np.arange(lookback, len(predicted_prices)+lookback), predicted_prices, color='red', label='Predicted Cryptocurrency Price')
plt.title('Cryptocurrency Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten

# Load the dataset
df = pd.read_csv('crypto_prices.csv')

# Convert the Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort the dataset by date
df = df.sort_values('Date')

# Select the 'Close' price for prediction
data = df['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define the sequence length (e.g., 60 days)
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Reshape X to be 3D (samples, timesteps, features) for the CNN+LSTM model
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential()

# Add CNN layers
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# Add LSTM layers
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Add Dense layer
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print model summary
model.summary()

# Split the data into training and testing sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predict on the test set
predicted_prices = model.predict(X_test)

# Inverse transform to get the actual prices
predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))

# Inverse transform of actual prices for comparison
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the predictions vs actual
plt.figure(figsize=(12, 6))
plt.plot(actual_prices, color='black', label='Actual Prices')
plt.plot(predicted_prices, color='green', label='Predicted Prices')
plt.title('Cryptocurrency Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(actual_prices, predicted_prices)
print(f"Mean Squared Error: {mse}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset (Replace 'crypto_data.csv' with your actual data file)
data = pd.read_csv('crypto_data.csv')
data = data[['Date', 'Close']]  # Using Date and Close price for simplicity
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

# Create input sequences for CNN
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 50  # Number of days to look back
X, y = create_sequences(data['Close'].values, seq_length)

# Split into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten

# Build the CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_length, 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.fit(X_train.reshape(-1, seq_length, 1), y_train, epochs=10, batch_size=32, validation_split=0.2)

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Extract features using the CNN model (or use raw features)
cnn_features = cnn_model.predict(X_train.reshape(-1, seq_length, 1))
cnn_features_test = cnn_model.predict(X_test.reshape(-1, seq_length, 1))

# Train the SVM model on CNN features
svm_model = SVR(kernel='rbf')
svm_model.fit(cnn_features, y_train)

# Predict and evaluate
y_pred = svm_model.predict(cnn_features_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

import matplotlib.pyplot as plt

plt.plot(y_test, label='True Prices')
plt.plot(y_pred, label='Predicted Prices')
plt.legend()
plt.show()
